In [1]:
PATH_PREFIX = '../../'
import sys
sys.path.append(PATH_PREFIX)

In [2]:
import torch
import os
from utils.weight_sharing import *
from data.imagenette import *
from data.utils.download import *
from data.utils.imagenet_utils import *

BATCH_SIZE = 32

In [3]:
net = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

dat = ImagenetteDataset(BATCH_SIZE, os.path.join(PATH_PREFIX, 'data/imagenette/'), val_split=0.3)


Using cache found in /home/coupekv/.cache/torch/hub/pytorch_vision_v0.10.0


In [4]:
print('test ', len(dat.test_dataset))
print('val', len(dat.valid_dataset))
print('train', len(dat.train_dataset))

test  2748
val 1177
train 9469


In [5]:
acc = get_accuracy(net, dat.test_dl, 'cpu', topk=1)
#acc - 0.01 * acc
acc

0.828966498374939

In [6]:
acc

0.828966498374939

In [7]:
net

MobileNetV2(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05,

In [8]:
ws_controller = WeightShare(net, lambda: get_accuracy(net, dat.test_dl, 'cpu', topk=1))
ws_controller.print_layers_info()

layer_name #weights #bias w_locked CR
features.0.0 864 0 False 1.00
features.1.conv.0.0 288 0 False 1.00
features.1.conv.1 512 0 False 1.00
features.2.conv.0.0 1536 0 False 1.00
features.2.conv.1.0 864 0 False 1.00
features.2.conv.2 2304 0 False 1.00
features.3.conv.0.0 3456 0 False 1.00
features.3.conv.1.0 1296 0 False 1.00
features.3.conv.2 3456 0 False 1.00
features.4.conv.0.0 3456 0 False 1.00
features.4.conv.1.0 1296 0 False 1.00
features.4.conv.2 4608 0 False 1.00
features.5.conv.0.0 6144 0 False 1.00
features.5.conv.1.0 1728 0 False 1.00
features.5.conv.2 6144 0 False 1.00
features.6.conv.0.0 6144 0 False 1.00
features.6.conv.1.0 1728 0 False 1.00
features.6.conv.2 6144 0 False 1.00
features.7.conv.0.0 6144 0 False 1.00
features.7.conv.1.0 1728 0 False 1.00
features.7.conv.2 12288 0 False 1.00
features.8.conv.0.0 24576 0 False 1.00
features.8.conv.1.0 3456 0 False 1.00
features.8.conv.2 24576 0 False 1.00
features.9.conv.0.0 24576 0 False 1.00
features.9.conv.1.0 3456 0 False 1.

In [9]:
ws_controller.model_layers[0].weight.dtype

torch.float32

In [10]:
len(ws_controller.model_layers)

53

In [11]:
ws_controller.share([70, 96, 103, 120, 120, 120, 31, 120, 96, 120, 120, 117, 12, 55, 120, 63, 23, 73, 120, 28, 74, 12, 55, 106, 120, 102, 120, 13, 68, 65, 48, 69, 120, 22, 120, 71, 21, 120, 120, 57, 53, 91, 120, 120, 18, 120, 93, 32, 63, 120, 120, 120, 114], verbose=True, prec_reduct=['f4' for _ in ws_controller.model_layers], minibatch_kmeans=False)

TypeError: WeightShare.share() got an unexpected keyword argument 'minibatch_kmeans'

In [ ]:
classes = json.load(open('../data/imagenette/imagenet_class_index.json'))
classes_arr = []

for i in range(len(classes.keys())):
    classes_arr.append(classes[f'{i}'])

output = net(dat.test_dl.__getitem__(1)[0].unsqueeze(0))

probabilities = torch.nn.functional.softmax(output[0], dim=0)

top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(classes_arr[top5_catid[i]], top5_prob[i].item(), top5_catid[i])

FileNotFoundError: [Errno 2] No such file or directory: '../data/imagenette/imagenet_class_index.json'